In [1]:
import os, json, polars as pl, numpy as np, lightgbm as lgb
from sklearn.model_selection import GroupKFold
from typing import List
from tqdm.auto import tqdm
from lightgbm.callback import CallbackEnv

C:\Users\idine\PycharmProjects\Avito_Test\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 42
np.random.seed(SEED)

BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, "Data")
TRAIN_FEATS_PATH = os.path.join(DATA_DIR, "train_feats2.parquet")
TEST_FEATS_PATH  = os.path.join(DATA_DIR, "test_feats2.parquet")

OUT_DIR = "out_lgbm"; os.makedirs(OUT_DIR, exist_ok=True)
TARGET = "item_contact"; GROUP = "query_id"; ID_COLS = ["query_id","item_id"]

In [3]:
assert os.path.exists(TRAIN_FEATS_PATH);
assert os.path.exists(TEST_FEATS_PATH)
train_feats = pl.read_parquet(TRAIN_FEATS_PATH)
test_feats = pl.read_parquet(TEST_FEATS_PATH)
print("[load] train_feats:", train_feats.shape, "| test_feats:", test_feats.shape)

[load] train_feats: (7781790, 72) | test_feats: (335348, 71)


In [4]:
# ## 2) Описание набора фичей и колонок

FEATURES = [
    # эмбеддинги
    "cos_q_title",

    # тексты
    "query_len", "title_len", "abs_len_diff",
    "overlap_q_title", "jaccard_q_title", "dice_q_title", "ratio_overlap_title",
    "title_contains_query", "has_query_text", "has_title",

    # категории/локации
    "same_cat", "same_mcat", "same_loc", "same_cat_loc", "triple_match",
    "freq_item_cat", "freq_item_mcat", "freq_item_loc", "is_query_mcat_missing",

    # цена
    "price_clip", "price_log1p", "price_rank_in_query", "price_z_in_query", "price_is_zero", "price_vs_median_query",

    # поведенческие
    "conv_known", "conv_filled", "conv_z_in_query", "conv_rank_in_query",

    # групповые
    "n_items_in_query", "max_cos_in_query", "cos_minus_max",

    # интеракции
    "same_cat__cos", "same_loc__cos", "conv__cos", "conv__same_loc", "price_rank__same_loc",
]

In [5]:
def sanitize_features(df: pl.DataFrame, features: list[str]) -> pl.DataFrame:
    fixes = []
    for c in features:
        if c not in df.columns: continue
        dt = df.schema[c]
        if dt in (pl.Float32, pl.Float64):
            fixes.append(
                pl.when(pl.col(c).is_finite()).then(pl.col(c)).otherwise(0.0).fill_null(0.0).cast(pl.Float32).alias(c))
        elif dt == pl.Boolean:
            fixes.append(pl.col(c).cast(pl.Int8).fill_null(0).alias(c))
        elif dt.is_integer():
            fixes.append(pl.col(c).fill_null(0).alias(c))
        else:
            fixes.append(pl.col(c).cast(pl.Float32).fill_null(0.0).alias(c))
    return df.with_columns(fixes)


In [6]:
def prepare_matrix(df: pl.DataFrame, features: List[str]) -> np.ndarray:
    return df.select([pl.col(c).cast(pl.Float32) for c in features]).fill_null(0.0).to_numpy()

In [7]:
train_feats = sanitize_features(train_feats, FEATURES + [TARGET, GROUP])
test_feats  = sanitize_features(test_feats,  FEATURES + ID_COLS)


In [8]:
y = train_feats[TARGET].to_numpy()
groups_vec = train_feats[GROUP].to_numpy()
X = prepare_matrix(train_feats, FEATURES)
X_test = prepare_matrix(test_feats, FEATURES)
print("[matrix] X:", X.shape, "y:", y.shape, "X_test:", X_test.shape)

[matrix] X: (7781790, 38) y: (7781790,) X_test: (335348, 38)


In [9]:
N_FOLDS = 5
gkf = GroupKFold(n_splits=N_FOLDS)
folds = [(tr_idx, va_idx) for tr_idx, va_idx in gkf.split(X, y, groups=groups_vec)]

In [10]:
def group_sizes_from_sorted_ids(ids: np.ndarray) -> np.ndarray:
    _, counts = np.unique(ids, return_counts=True); return counts

class TQDMCallback:
    def __init__(self, total_rounds: int, desc: str="boosting"): self.t=total_rounds; self.desc=desc; self.p=None
    def __call__(self, env: CallbackEnv):
        if self.p is None:
            end=getattr(env,"end_iteration",self.t); beg=getattr(env,"begin_iteration",0)
            from tqdm.auto import tqdm as _tqdm; self.p=_tqdm(total=end-beg, desc=self.desc, leave=False)
        self.p.update(1)
        if env.iteration+1>=self.t or (env.iteration+1)>=getattr(env,"end_iteration",self.t): self.p.close(); self.p=None

def calc_dcg_at_k(v: np.ndarray, k: int = 10) -> float:
    w = 0.97 ** np.arange(len(v)); return (v * w)[:k].sum()
def calc_ndcg_at_k(labels: np.ndarray, preds: np.ndarray, groups: np.ndarray, k: int = 10) -> float:
    order = np.argsort(groups, kind="mergesort"); labels, preds, groups = labels[order], preds[order], groups[order]
    _, counts = np.unique(groups, return_counts=True); s=0; lst=[]
    for c in counts:
        sl = slice(s, s+c); l, p = labels[sl], preds[sl]; idx = np.argsort(-p, kind="mergesort")
        idcg = calc_dcg_at_k(np.sort(l)[::-1], k) + 1e-12; lst.append(calc_dcg_at_k(l[idx], k) / idcg); s += c
    return float(np.mean(lst))

In [11]:
params = dict(objective="lambdarank", metric="ndcg", ndcg_eval_at=[10],
              learning_rate=0.01, num_leaves=63, min_data_in_leaf=200,
              feature_fraction=0.85, bagging_fraction=0.8, bagging_freq=1,
              lambda_l2=1.0, verbose=-1, seed=SEED, num_threads=0, device="gpu")

num_boost_round=1200; early_stopping_rounds=80

In [12]:
oof_lgb = np.zeros(len(y), dtype=np.float32)
test_pred_lgb = np.zeros(len(X_test), dtype=np.float32)

for fold, (tr_idx, va_idx) in enumerate(folds, 1):
    tr_s = tr_idx[np.argsort(groups_vec[tr_idx], kind="mergesort")]
    va_s = va_idx[np.argsort(groups_vec[va_idx], kind="mergesort")]
    dtr = lgb.Dataset(X[tr_s], label=y[tr_s], group=group_sizes_from_sorted_ids(groups_vec[tr_s]), feature_name=FEATURES)
    dva = lgb.Dataset(X[va_s], label=y[va_s], group=group_sizes_from_sorted_ids(groups_vec[va_s]), feature_name=FEATURES)
    try:
        model = lgb.train(params, dtr, valid_sets=[dva], num_boost_round=num_boost_round,
                          callbacks=[TQDMCallback(num_boost_round, f"LGB fold {fold}"),
                                     lgb.early_stopping(early_stopping_rounds, verbose=False),
                                     lgb.log_evaluation(100)])
    except lgb.basic.LightGBMError:
        params["device"]="cpu"
        model = lgb.train(params, dtr, valid_sets=[dva], num_boost_round=num_boost_round,
                          callbacks=[TQDMCallback(num_boost_round, f"LGB fold {fold} (cpu)"),
                                     lgb.early_stopping(early_stopping_rounds, verbose=False),
                                     lgb.log_evaluation(100)])
    oof_lgb[va_s] = model.predict(X[va_s], num_iteration=model.best_iteration)
    test_pred_lgb += model.predict(X_test, num_iteration=model.best_iteration) / N_FOLDS
    print(f"fold {fold}: ndcg@10={calc_ndcg_at_k(y[va_s], oof_lgb[va_s], groups_vec[va_s]):.5f}")

print("[LGBM] CV ndcg@10:", calc_ndcg_at_k(y, oof_lgb, groups_vec))


LGB fold 1:   8%|▊         | 100/1200 [00:27<05:29,  3.34it/s]

[100]	valid_0's ndcg@10: 0.877316


LGB fold 1:  17%|█▋        | 200/1200 [00:54<04:24,  3.78it/s]

[200]	valid_0's ndcg@10: 0.879173


LGB fold 1:  25%|██▌       | 300/1200 [01:20<03:43,  4.03it/s]

...

LGB fold 5:  83%|████████▎ | 1001/1200 [03:27<00:33,  5.89it/s]

[1000]	valid_0's ndcg@10: 0.883479


LGB fold 5:  92%|█████████▏| 1101/1200 [03:42<00:16,  5.94it/s]

[1100]	valid_0's ndcg@10: 0.883619


[1200]	valid_0's ndcg@10: 0.883694
fold 5: ndcg@10=0.31020
[LGBM] CV ndcg@10: 0.30976922610916974


In [13]:
pl.DataFrame(
    {"query_id": train_feats["query_id"], "item_id": train_feats["item_id"], "oof_lgb": oof_lgb}).write_parquet(
    os.path.join(OUT_DIR, "oof_lgb.parquet"))
pl.DataFrame(
    {"query_id": test_feats["query_id"], "item_id": test_feats["item_id"], "pred_lgb": test_pred_lgb}).write_parquet(
    os.path.join(OUT_DIR, "test_pred_lgb.parquet"))
print("[save] out_lgbm/oof_lgb.parquet & out_lgbm/test_pred_lgb.parquet")

[save] out_lgbm/oof_lgb.parquet & out_lgbm/test_pred_lgb.parquet
